In [2]:
from openai import AzureOpenAI

# load the API key from env file
import os
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),
  api_version="2024-08-01-preview"
)
model = "gpt-4o"

In [4]:
from functions import get_llm_response

messages = [{"role": "user", "content": "What is the capital of France?"}]

print(get_llm_response(client, model, messages))

The capital of France is Paris.


## Using json response

Useful when doing classification or connecting LLM output to deterministic functions.

You must at least mention "json" in the system prompt.

More info: https://platform.openai.com/docs/guides/structured-outputs

In [5]:
from functions import get_llm_response_in_json

messages = [
    {"role": "system", "content": "You must return a json object with the following keys: 'capital', 'country', 'continent'."},
    {"role": "user", "content": "What is the capital of France?"}
    ]

print(get_llm_response_in_json(client, model, messages))

{
  "capital": "Paris",
  "country": "France",
  "continent": "Europe"
}


## Using images

You can either use a url of a public image or first encode the image and send it in the same manner. 

More info: https://platform.openai.com/docs/guides/vision

In [6]:
from functions import encode_image

# Path to your image
image_path = "data/thumbs up.jpg"

# Getting the base64 string
base64_image = encode_image(image_path)


messages=[
{
    "role": "user",
    "content": [
    {
        "type": "text",
        "text": "What is in this image?",
    },
    {
        "type": "image_url",
        "image_url": {
        "url":  f"data:image/jpeg;base64,{base64_image}"
        },
    },
    ],
}
]

# Now you can use these messages with your LLM client
print(get_llm_response(client, model, messages))

The image shows a person giving a thumbs-up gesture. There is a computer monitor visible on the right side of the image. Additionally, there is a signature or text that reads "Brent Rambo" at the bottom of the image.


## Function calling

Beyond generating text content, you can provide GPT with your own python functions and instruct it to use them when appropriate. GPT will provide all the variables for the function based on it's best judgement. 

More info: https://platform.openai.com/docs/guides/function-calling

In [7]:
from functions import function_to_schema, get_llm_response_with_functions
import json

# Define your functions with a docstring and a return type
def print_string(s: str) -> None:
    """
    Print a string
    """
    print(s)

def add_two_numbers(a: int, b: int) -> int:
    """
    Add two numbers
    """
    return a + b

# create a map of function names to functions
tool_map = {
    "print_string": print_string,
    "add_two_numbers": add_two_numbers
}

# Transform the function into GPT digestible schema
tool_schemas = [function_to_schema(print_string),
                function_to_schema(add_two_numbers)]

messages = [
    {"role": "system", "content": "You must use provided functions when appropriate."},
    {"role": "user", "content": "What is 19870 + 16890?"}]

response = get_llm_response_with_functions(client, model, messages, tool_schemas)

# tool requesting message has to precede the function result message 
# if you want to send the request again in a loop
messages.append(response.choices[0].message)

if response.choices[0].message.tool_calls:
    for tool_call in response.choices[0].message.tool_calls:
        function_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        tool_response = tool_map[function_name](**arguments)
        messages.append({"role": "tool", "content": tool_response})
        print(tool_response)
        

36760
